In [1]:
from bisect import bisect
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import DataLoader
from dataset import uniformDataset
from model import MLP
from train import train_model, evaluate_model
from test import test_model

cuda = torch.cuda.is_available()
num_workers = 8 if cuda else 0 
device = torch.device("cuda" if cuda else "cpu")
verbose = True

In [2]:
# TODO: real train/dev/test dataset
# TODO: training curves
# TODO: concatenate context

In [3]:
print("torch version: %s" % torch.__version__)
print("np version: %s" % np.__version__)
print("__debug__: %s" % __debug__)
print("cuda: %s" % cuda)
print("num_workers: %s" % num_workers)
print("device: %s" % device)
print("verbose: %s" % verbose)

torch version: 1.3.1
np version: 1.17.4
__debug__: True
cuda: False
num_workers: 0
device: cpu
verbose: True


In [4]:
path_trainx = "../data/train.npy"
path_trainy = "../data/train_labels.npy"
path_evalx = "../data/dev.npy"
path_evaly = "../data/dev_labels.npy"
path_testx = "../data/test.npy"

In [5]:
evalx = np.load(path_evalx, allow_pickle=True)
evaly = np.load(path_evaly, allow_pickle=True)

testx = np.load(path_testx, allow_pickle=True)

In [9]:
# data loader
print("loading data...")

train_dataset = uniformDataset(evalx, evaly) # TODO: change this to real train dataset
eval_dataset = uniformDataset(evalx, evaly) # TODO: change this to real train dataset
test_dataset = uniformDataset(testx, is_test=True)

if verbose:
    print("train size: X:({}, {}) Y:({}, 1)".format(len(train_dataset), len(train_dataset[0][0]), len(train_dataset)))
    print("eval size: X:({}, {}) Y:({}, 1)".format(len(eval_dataset), len(eval_dataset[0][0]), len(eval_dataset)))
    print("test size: X:({}, {})".format(len(test_dataset), len(test_dataset[0])))

batch_size = len(train_dataset) // train_dataset.utter_num # average #frames in one utterance in train dataset

train_loader = DataLoader(
    train_dataset, # The dataset
    batch_size=batch_size,      # Batch size
    shuffle=True,      # Shuffles the dataset at every epoch
    pin_memory=True,   # Copy data to CUDA pinned memory
    num_workers=num_workers      # Number of worker processes for loading data.
                       )

eval_loader = DataLoader(
    eval_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=num_workers
)

test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    shuffle=False,
    pin_memory=True,
    num_workers=num_workers
)

loading data...
train size: X:(675836, 40) Y:(675836, 1)
eval size: X:(675836, 40) Y:(675836, 1)
test size: X:(223592, 40)


In [10]:
# model
print("building model...")

size_list = [40, 256, 138] # TODO: change this
model = MLP(size_list, nn.ReLU)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

building model...


In [11]:
# training
print("training...")
epochs = 2
train_losses, eval_losses, eval_accs = \
    train_model(model, epochs, train_loader, eval_loader, criterion, optimizer, device)

training...
Training Loss:  5.30270888391784 Time:  12.655793905258179 s
evaluate Loss:  3.6799234445261804
evaluate Accuracy:  14.984848395172792 %
Training Loss:  3.4598245200625772 Time:  10.821533918380737 s
evaluate Loss:  3.346704479134375
evaluate Accuracy:  19.899502246107044 %


In [12]:
print("predicting...")
predicts = test_model(model, test_loader, device, save=True)
print("finished")

predicting...
